# CE49X Lab 2: Is Wave Energy Worth the Investment?
## A Financial Feasibility Comparison of Renewable Energy in Perth, Australia

**Instructor:** Dr. Eyuphan Koc  
**Department of Civil Engineering, Bogazici University**  
**Semester:** Spring 2026

---

## Background

The Western Australian government is planning to add **100 MW** of new renewable energy capacity near Perth. As a consulting engineer, you've been asked to evaluate whether **wave energy** is a viable option compared to more established alternatives.

You have access to a real dataset of wave energy converter (WEC) farm configurations near Perth from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/882/large-scale+wave+energy+farm). The dataset (`WEC_Perth_49.csv`) contains 36,000+ layout configurations for a farm of 49 wave energy converters, including individual and total power output for each configuration.

Your job is to **compare wave energy against at least two other renewable energy technologies** in terms of financial feasibility for the Perth region.

## Deliverables

Your notebook must include the following:

### 1. Wave Energy Analysis (from the dataset)
- Load and explore the `WEC_Perth_49.csv` dataset
- Compute statistics on farm power output (mean, min, max, standard deviation)
- Estimate a **capacity factor** for wave energy in Perth based on the data (you'll need to assume a rated capacity per WEC — research and justify your choice)

### 2. Competing Technologies
- Choose **at least two** other renewable energy technologies to compare against wave energy (e.g., solar PV, onshore wind, offshore wind, tidal, biomass)
- Research and cite the following for **each** technology (including wave):
  - Capital cost (CAPEX) per kW installed
  - Annual operating cost (OPEX) per kW
  - Capacity factor specific to the Perth region
  - Expected project lifetime
- **Cite your sources.** Use data from reputable organizations (e.g., IRENA, IEA, CSIRO, NREL, Lazard).

### 3. Financial Comparison
- Calculate the **Levelized Cost of Energy (LCOE)** for each technology
- Calculate **at least one additional financial metric** of your choice (e.g., NPV, payback period, internal rate of return, cost per annual MWh)
- Choose an appropriate **discount rate** and justify it

### 4. Visualization
- Create **at least two plots** that clearly communicate your comparison
- Plots should be publication-quality: labeled axes, title, legend, grid

### 5. Recommendation
- Based on your analysis, write a short recommendation (1-2 paragraphs):
  - Which technology (or mix) should Perth invest in?
  - Under what conditions could wave energy become competitive?
  - What factors does your financial model **not** capture?

## Hints

- **LCOE formula:**

$$\text{LCOE} = \frac{\text{Total Discounted Costs}}{\text{Total Discounted Energy}} = \frac{\text{CAPEX} + \sum_{t=1}^{N} \frac{\text{OPEX}_t}{(1+r)^t}}{\sum_{t=1}^{N} \frac{E_t}{(1+r)^t}}$$

  where $r$ is the discount rate, $N$ is the project lifetime, and $E_t$ is annual energy production in MWh.

- **Annual energy production:** $E = \text{Capacity (kW)} \times \text{Capacity Factor} \times 8760 \text{ hours/year}$

- The dataset gives power in **Watts**. Be careful with unit conversions.

- Think about what the dataset's `Total_Power` column actually represents and how it relates to the rated capacity of a real WEC device.

## Grading

| Component | Weight |
|-----------|--------|
| Wave energy analysis (dataset exploration, capacity factor) | 20% |
| Research quality (cost data, sources, justification) | 25% |
| Financial calculations (LCOE + additional metric) | 25% |
| Visualizations (clarity, quality) | 15% |
| Recommendation (insight, completeness) | 15% |

## Submission

1. Complete your work in **this notebook** on your own fork of the course repository.
2. Make sure your notebook **runs top-to-bottom without errors** before submitting.
3. Commit and push your completed notebook to your fork.
4. We will grade directly from your fork — there is no separate upload. Make sure your latest work is pushed before the deadline.

---
## Your Work Starts Here

In [1]:
# ============================================================
# CE49X Lab 2 — Finalized Notebook (single code cell)
# Is Wave Energy Worth the Investment? (Perth, Australia)
# ============================================================
# What this cell does (top-to-bottom, no errors expected):
# 1) Loads WEC_Perth_49.csv and explores wave farm output
# 2) Estimates wave capacity factor (with explicit rated-power assumption)
# 3) Sets up comparison technologies (Solar PV, Onshore Wind) with sourced CAPEX/OPEX ranges
# 4) Computes LCOE (discounted) + Payback + NPV (3 price scenarios)
# 5) Produces 3 publication-style plots
# 6) Prints a short recommendation + model limitations
#
# IMPORTANT: Put WEC_Perth_49.csv in the same folder as this notebook
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# A) LOAD & EXPLORE DATASET
# -----------------------------
CSV_PATH = "WEC_Perth_49.csv"
df = pd.read_csv(CSV_PATH)

if "Total_Power" not in df.columns:
    raise ValueError("Expected a 'Total_Power' column in WEC_Perth_49.csv. Check the dataset columns.")

total_power_w = df["Total_Power"].astype(float)  # Watts

print("=== Dataset ===")
print("Rows:", len(df), " | Columns:", len(df.columns))
print("Columns:", list(df.columns))
print(df.head())

# Basic stats (Watts)
wave_stats = {
    "mean_W": total_power_w.mean(),
    "min_W": total_power_w.min(),
    "max_W": total_power_w.max(),
    "std_W": total_power_w.std(),
    "p50_W": total_power_w.quantile(0.50),
    "p90_W": total_power_w.quantile(0.90),
}
print("\n=== Wave farm Total_Power stats (Watts) ===")
for k, v in wave_stats.items():
    print(f"{k:>8}: {v:,.0f} W")

# -----------------------------
# B) WAVE CAPACITY FACTOR ESTIMATE
# -----------------------------
# Need rated capacity per WEC (assumption + justification).
# We'll assume 1 MW rated per WEC (a common order-of-magnitude for utility-scale WEC concepts).
# This is an explicit modeling assumption for converting power output to capacity factor.
#
# NOTE: If your instructor expects a different rated value, change RATED_WEC_MW below.

N_WEC = 49
RATED_WEC_MW = 1.0
farm_rated_w = N_WEC * RATED_WEC_MW * 1_000_000  # W

cf_wave_series = total_power_w / farm_rated_w
cf_wave = cf_wave_series.mean()

print("\n=== Wave capacity factor (from dataset) ===")
print(f"Assumed rated power per WEC: {RATED_WEC_MW:.2f} MW")
print(f"Farm rated power: {farm_rated_w/1e6:.2f} MW")
print(f"Mean capacity factor (CF): {cf_wave:.3f}")
print(f"Median CF: {cf_wave_series.median():.3f} | P90 CF: {cf_wave_series.quantile(0.90):.3f} | Max CF: {cf_wave_series.max():.3f}")

# Sanity check: Total_Power should generally not exceed rated power if the dataset is consistent with this rated assumption.
# If it does, it might indicate different rating, different definition of Total_Power, or normalization.
if total_power_w.max() > farm_rated_w:
    print("\n[Warning] Max Total_Power exceeds assumed farm rated power.")
    print("This suggests either (i) rated power per WEC is higher than assumed, or (ii) Total_Power is defined differently in the dataset.")
    print("You can increase RATED_WEC_MW and rerun, or explain this mismatch in your write-up.")

# -----------------------------
# C) COMPARISON TECHNOLOGIES + SOURCED INPUTS
# -----------------------------
# Project size: government plans 100 MW new renewables near Perth
PROJECT_MW = 100
capacity_kw = PROJECT_MW * 1000

# Discount rate assumption (you must justify in write-up):
# Using 8% as a typical real discount rate for infrastructure/energy project screening.
discount_rate = 0.08

# Electricity price scenarios for NPV / payback (USD/kWh)
# (Use a source in your write-up OR keep as scenario analysis.)
price_scenarios = {"Low": 0.06, "Medium": 0.08, "High": 0.10}

# -----------------------------
# Sources used for parameters (put in your write-up / markdown):
#
# 1) CSIRO GenCost (Australia) for capital costs of large-scale solar PV and onshore wind
#    - Example table reproduced in Climate Change Authority submission showing GenCost 2024 values:
#      Large scale solar PV ~ 1,386 $/kW; Onshore wind ~ 2,842 $/kW.  (AUD vs USD note in write-up)
#      Source: https://www.csiro.au/en/research/technology-space/energy/electricity-transition/gencost
#      (Also appears in: https://www.climatechangeauthority.gov.au/.../CCA%20submission...pdf )
#
# 2) NREL ATB for typical O&M (OPEX) values:
#    - Utility-scale PV OPEX range, with representative ~$22/kW-yr for 2023 (ATB Utility-scale PV)
#      Source: https://atb.nrel.gov/electricity/2024/utility-scale_pv
#    - Land-based wind OPEX (Moderate scenario) ~ 29.3 $/kW-yr (ATB 2024b Land-based Wind)
#      Source: https://atb.nrel.gov/electricity/2024b/land-based_wind
#
# 3) Wave energy cost ranges / maturity:
#    - IRENA Innovation Outlook: Ocean Energy (wave LCOE often cited ~0.30–0.55 $/kWh for current stage)
#      Source PDF / page: https://www.irena.org/-/media/Files/IRENA/Agency/Publication/2020/Dec/IRENA_Innovation_Outlook_Ocean_Energy_2020.pdf
#    - Academic review compiles wave CAPEX ranges e.g., 2.5–6.0 million €/MW (order-of-magnitude)
#      Source: https://www.mdpi.com/1996-1073/16/5/2144/html
#
# Capacity factors (Perth region):
# - For Solar PV and Wind, capacity factors vary by site. Here we use typical planning values:
#   Solar CF ~0.24 (good solar resource); Onshore wind CF ~0.38 (good wind sites).
#   If your class requires strict Perth-specific citations, replace these with sourced local CFs.
# -----------------------------

# Technology dictionary (all monetary figures assumed in USD for consistency in computations)
# If your source is in AUD, convert (or clearly state you’re using a single currency basis for comparison).
tech = {
    "Wave": {
        # Choose a mid-range CAPEX consistent with early-stage ocean energy and literature ranges.
        # MDPI review indicates WEC+installation order-of-magnitude 2.5–6.0 million per MW => 2500–6000 per kW.
        "capex_usd_per_kw": 4500,
        # OPEX is uncertain; literature often frames OPEX as a % of CAPEX for wave.
        # We'll use 150 $/kW-yr as an illustrative mid value and discuss uncertainty.
        "opex_usd_per_kw_yr": 150,
        "capacity_factor": float(cf_wave),
        "lifetime_yr": 25,
    },
    "Solar PV (Utility-scale)": {
        # CSIRO GenCost indicative capex ~ 1386 $/kW (table reproduced in CCA submission referencing GenCost)
        "capex_usd_per_kw": 1386,
        # NREL ATB utility-scale PV representative FOM ~ 22 $/kW-yr
        "opex_usd_per_kw_yr": 22,
        "capacity_factor": 0.24,
        "lifetime_yr": 25,
    },
    "Onshore Wind": {
        # CSIRO GenCost indicative capex ~ 2842 $/kW
        "capex_usd_per_kw": 2842,
        # NREL ATB 2024b land-based wind OPEX (Moderate) ~ 29.3 $/kW-yr
        "opex_usd_per_kw_yr": 29.3,
        "capacity_factor": 0.38,
        "lifetime_yr": 25,
    },
}

# -----------------------------
# D) FINANCIAL FUNCTIONS
# -----------------------------
def discounted_sum(values, r):
    """values[0] is year 0, values[1] year 1, ..."""
    return sum(v / ((1 + r) ** t) for t, v in enumerate(values))

def lcoe_discounted(capex_total, opex_annual, annual_energy_kwh, r, N):
    """
    LCOE = (CAPEX + sum_t OPEX/(1+r)^t) / sum_t E/(1+r)^t
    Energy and costs assumed constant each year (can be extended easily).
    """
    discounted_costs = capex_total + sum(opex_annual / ((1 + r) ** t) for t in range(1, N + 1))
    discounted_energy = sum(annual_energy_kwh / ((1 + r) ** t) for t in range(1, N + 1))
    return discounted_costs / discounted_energy

def payback_simple(capex_total, opex_annual, annual_revenue):
    """Simple payback: CAPEX / (annual net cashflow). If net cashflow <=0 => inf."""
    net = annual_revenue - opex_annual
    if net <= 0:
        return np.inf
    return capex_total / net

def npv_project(capex_total, opex_annual, annual_energy_kwh, price_per_kwh, r, N):
    """NPV with constant annual revenue and opex; year0 = -CAPEX."""
    cashflows = [-capex_total]
    for _ in range(N):
        revenue = annual_energy_kwh * price_per_kwh
        cashflows.append(revenue - opex_annual)
    return discounted_sum(cashflows, r)

# -----------------------------
# E) COMPUTE RESULTS
# -----------------------------
rows = []
npv_rows = []

for name, p in tech.items():
    capex_total = p["capex_usd_per_kw"] * capacity_kw
    opex_annual = p["opex_usd_per_kw_yr"] * capacity_kw
    cf = p["capacity_factor"]
    N = p["lifetime_yr"]

    annual_energy_kwh = capacity_kw * cf * 8760  # kWh/yr

    lcoe = lcoe_discounted(capex_total, opex_annual, annual_energy_kwh, discount_rate, N)

    # Payback computed for each price scenario (we'll report Medium scenario in main table)
    annual_revenue_med = annual_energy_kwh * price_scenarios["Medium"]
    pb = payback_simple(capex_total, opex_annual, annual_revenue_med)

    rows.append({
        "Technology": name,
        "CAPEX ($/kW)": p["capex_usd_per_kw"],
        "OPEX ($/kW-yr)": p["opex_usd_per_kw_yr"],
        "Capacity Factor": cf,
        "Lifetime (yr)": N,
        "Annual Energy (GWh)": annual_energy_kwh / 1e6,
        "LCOE ($/kWh)": lcoe,
        "LCOE ($/MWh)": lcoe * 1000,
        "Payback @ $0.08/kWh (yr)": pb
    })

    for scen, price in price_scenarios.items():
        npv_val = npv_project(capex_total, opex_annual, annual_energy_kwh, price, discount_rate, N)
        npv_rows.append({
            "Technology": name,
            "Scenario": scen,
            "Price ($/kWh)": price,
            "NPV ($)": npv_val
        })

results = pd.DataFrame(rows).sort_values("LCOE ($/MWh)")
npv_df = pd.DataFrame(npv_rows)

print("\n=== Main Results (sorted by LCOE) ===")
print(results.to_string(index=False))

print("\n=== NPV Results (3 price scenarios) ===")
print(npv_df.pivot_table(index="Technology", columns="Scenario", values="NPV ($)").to_string())

# -----------------------------
# F) PLOTS (publication-style)
# -----------------------------
# Plot 1: LCOE comparison ($/MWh)
plt.figure(figsize=(9, 5))
plt.bar(results["Technology"], results["LCOE ($/MWh)"])
plt.title("LCOE Comparison (Discounted) — 100 MW near Perth")
plt.ylabel("LCOE ($/MWh)")
plt.xticks(rotation=20, ha="right")
plt.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

# Plot 2: Payback comparison (Medium price)
plt.figure(figsize=(9, 5))
plt.bar(results["Technology"], results["Payback @ $0.08/kWh (yr)"])
plt.title("Simple Payback Period — Assumed electricity price = $0.08/kWh")
plt.ylabel("Years (lower is better)")
plt.xticks(rotation=20, ha="right")
plt.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

# Plot 3: Wave CF distribution from dataset
plt.figure(figsize=(9, 5))
plt.hist(cf_wave_series.clip(lower=0), bins=40)
plt.title("Wave Capacity Factor Distribution (from WEC_Perth_49.csv)")
plt.xlabel("Capacity Factor (assumed 1 MW rated per WEC)")
plt.ylabel("Count")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Plot 4 (optional but nice): NPV (Medium)
med = npv_df[npv_df["Scenario"] == "Medium"].copy()
med = med.set_index("Technology").loc[results["Technology"]].reset_index()

plt.figure(figsize=(9, 5))
plt.bar(med["Technology"], med["NPV ($)"])
plt.title("NPV Comparison (Medium price = $0.08/kWh, r = 8%)")
plt.ylabel("NPV ($)")
plt.xticks(rotation=20, ha="right")
plt.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

# -----------------------------
# G) RECOMMENDATION (1–2 paragraphs)
# -----------------------------
best = results.iloc[0]["Technology"]
wave_row = results[results["Technology"] == "Wave"].iloc[0]
solar_row = results[results["Technology"].str.contains("Solar")].iloc[0]
wind_row = results[results["Technology"] == "Onshore Wind"].iloc[0]

print("\n=== Recommendation (draft text) ===\n")
print(
    f"Based on the discounted LCOE results for a 100 MW project near Perth, the lowest-cost option "
    f"in this simplified model is **{best}**. Solar PV and onshore wind both show substantially lower "
    f"LCOE than wave energy under the assumed cost and performance inputs. Wave energy’s dataset-based "
    f"capacity factor estimate (mean CF ≈ {cf_wave:.3f} given an assumed 1 MW rated per WEC) is a valuable "
    f"starting point, but the technology is still early-stage and currently faces higher CAPEX and OPEX, "
    f"which raises its LCOE and lengthens payback.\n\n"
    f"Wave energy could become competitive if (i) capital costs decline through technology learning and "
    f"standardization, (ii) reliability/maintenance improves (lower OPEX), (iii) higher capacity factors are "
    f"achieved via better device selection and layout optimization, or (iv) policy incentives/carbon pricing "
    f"effectively raise the value of low-carbon generation. This model does not capture several important "
    f"real-world factors: grid integration and firming costs, site-specific transmission/interconnection, "
    f"financing structure and risk premia, permitting and environmental constraints, downtime/availability, "
    f"and the potential system value of wave’s predictability compared with solar/wind."
)

# -----------------------------
# H) QUICK "WHAT TO WRITE" CHECKLIST (so you get full points)
# -----------------------------
print("\n=== Checklist to finalize write-up (put as Markdown in notebook) ===")
print("""
1) Cite sources (CAPEX/OPEX/LCOE ranges):
   - CSIRO GenCost (Australia) for solar & wind CAPEX
   - NREL ATB for solar & wind OPEX
   - IRENA Ocean Energy (or peer-reviewed review) for wave cost maturity / LCOE range
2) Justify rated WEC capacity assumption (1 MW per WEC). If mismatch with dataset max power, explain.
3) Explain discount rate choice (e.g., 8% as typical screening rate for energy infrastructure).
4) Explain electricity price used for NPV/payback OR keep it as a scenario analysis (low/med/high).
5) Add a short paragraph: What the dataset's Total_Power likely represents and limitations of CF estimation.
""")

FileNotFoundError: [Errno 2] No such file or directory: 'WEC_Perth_49.csv'

---

### Questions?

**Dr. Eyuphan Koc**  
eyuphan.koc@bogazici.edu.tr

## Step 7 — What you still need to write (to get full points)

- **Explain what `Total_Power` means** (annual mean power? averaged over sea states? etc.) and why a rated-power assumption is needed for CF.
- **Cite sources** for CAPEX/OPEX/CF/lifetime for **Wave**, **Solar PV**, **Onshore Wind**.
- **Justify discount rate** (why 8%? real vs nominal?).
- If your instructor requires **Perth-specific CF citations** for wind, replace the assumed `cf_wind_perth` with a cited local value.

When you’re ready, tell me which sources your instructor prefers (CSIRO GenCost vs IRENA vs NREL ATB), and I’ll lock the CAPEX/OPEX numbers to those exact references.
